## Visualization of tracks

In [1]:
# import cairosvg
# cairosvg.svg2png(url='output/Global_Reconstruction/global_relative_rotation_pose_graph.svg', 
# write_to='output/Global_Reconstruction/global_relative_rotation_pose_graph.png')

In [3]:
import open3d as o3d
import numpy as np

In [4]:
path = "output_drone/reconstruction_sequential/colorized.ply"
pcd = o3d.io.read_point_cloud(path) # Read the point cloud
o3d.visualization.draw_geometries([pcd])


Alpha shape

In [4]:
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(
        pcd, alpha=0.03)

In [5]:
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

Use point_cloud_ball_pivoting for meshing

In [5]:
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
        radius=0.1, max_nn=30)) # max_nn = 30 default

In [11]:
radii = [0.005, 0.01, 0.02, 0.04]
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
        pcd, o3d.utility.DoubleVector(radii))

In [12]:
o3d.visualization.draw_geometries([rec_mesh])

Use Poisson for meshing

In [ ]:
# # Another option to estimate normals
# # or you might want to flip the normals to make them point outward, not mandatory
# pcd.normals = o3d.utility.Vector3dVector( - np.asarray(pcd.normals))

point cloud poisson often needs to crop some parts out

In [6]:
bbox = pcd.get_axis_aligned_bounding_box()

rec_mesh2 = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth = 15, width = 0, scale = 1.1, linear_fit = True)[0]

rec_mesh2 = rec_mesh2.crop(bbox)

In [26]:
o3d.visualization.draw_geometries([rec_mesh2])

AttributeError: module 'open3d.visualization' has no attribute 'jupyter_draw_geometries'

In [5]:
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
        radius=0.1, max_nn=30)) # max_nn = 30 default

# estimate radius for rolling ball
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 5 * avg_dist   

mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
           pcd,
           o3d.utility.DoubleVector([radius, radius * 2]))



Downsample the result to an acceptable number of triangles, i.g. 100k
Additionally, in case mesh present some weird artifcats, below 2 commands to ensure its consistency.

In [7]:
mesh = rec_mesh2 

dec_mesh = mesh.simplify_quadric_decimation(100000)

dec_mesh.remove_duplicated_triangles()
dec_mesh.remove_non_manifold_edges()


TriangleMesh with 49279 points and 98694 triangles.

By using trimesh, visualization in Jupyter notebook is available. (It uses pyglet)

In [ ]:
o3d.visualization.draw_geometries([mesh])

In [14]:
import trimesh
mesh = dec_mesh
# create the triangular mesh with the vertices and faces from open3d
tri_mesh = trimesh.Trimesh(np.asarray(mesh.vertices), np.asarray(mesh.triangles),
                          vertex_normals=np.asarray(mesh.vertex_normals) ,vertex_colors=np.asarray(mesh.vertex_colors))

tri_mesh.show()

mesh[0]

In [6]:
import open3d as o3d
path = "/home/hexaburbach/Downloads/Meshroom-2021.1.0-linux-cuda10/Meshroom-2021.1.0-av2.4.0-centos7-cuda10.2/MeshroomCache/Texturing/1d6250ed17a67cef7f525b719fa83c68520267f0/texturedMesh.obj"

textured_mesh = o3d.io.read_triangle_mesh(path)



In [8]:
o3d.visualization.draw_geometries([textured_mesh])